In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import datetime as dt

In [2]:
gfd1 = pd.read_csv("...\\GFD-1\\FundPrice.txt", sep="\t", encoding='cp1252', parse_dates=["PriceDate"])
gfd1.head()

,LipperID,PriceDate,PriceCode,PriceType,PriceCurrency,PriceValueLC,Estimate
0,65006437,2017-08-17,D,N,CZK,9013.99,NaN
1,65006437,2017-08-16,D,N,CZK,9020.49,NaN
2,65006437,2017-08-15,D,N,CZK,8933.91,E
3,65006437,2017-08-14,D,N,CZK,8933.91,NaN
4,65006437,2017-08-11,D,N,CZK,9044.57,NaN


In [3]:
cfg = pd.read_csv("...\\CFG\\FundPrice.txt", sep="\t", parse_dates=["PriceDate"])
cfg_cols = cfg.columns
cfg.head()

,LipperID,PriceDate,PriceCode,PriceType,PriceCurrency,PriceValueLC,Estimate
0,40000041,2017-05-22,D,N,USD,10.48,NaN
1,40000041,2017-05-23,D,N,USD,10.49,NaN
2,40000041,2017-05-24,D,N,USD,10.49,NaN
3,40000041,2017-05-25,D,N,USD,10.50,NaN
4,40000041,2017-05-26,D,N,USD,10.51,NaN


In [4]:
cfg.equals(gfd1)

False

In [5]:
print("Missing in GFD-1: " + str(len(cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"])))
missing_in_gfd1_df = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]
missing_in_gfd1 = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"]


print("Missing in CFG: " + str(len(gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"])))
missing_in_cfg_df = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]
missing_in_cfg = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"]

universe_diffs = pd.concat([missing_in_cfg, missing_in_gfd1])

Missing in GFD-1: 10241
Missing in CFG: 0


In [ ]:
on=["LipperID", "PriceDate", "PriceCode", "PriceType"]
df = cfg.merge(gfd1, how='outer', on=on, suffixes=("_CFG", "_GFD-1"))
df = df[~df["LipperID"].isin(universe_diffs)]
df = df.fillna("-")
del cfg
del gfd1
df.sample(n=5)

,LipperID,PriceDate,PriceCode,PriceType,PriceCurrency_CFG,PriceValueLC_CFG,Estimate_CFG,PriceCurrency_GFD-1,PriceValueLC_GFD-1,Estimate_GFD-1
3792087,68231616,2017-08-04,D,N,EUR,116.81,-,EUR,116.81,-
3564457,68209792,2017-05-10,D,N,EUR,10.56,-,EUR,10.56,-
1635076,65107613,2017-05-10,D,N,EUR,210.98,-,EUR,210.98,-
5027076,68340033,2017-08-11,D,N,SGD,10.35,-,SGD,10.35,-
3669726,68221396,2017-05-16,D,N,USD,102.57,-,USD,102.57,-


In [ ]:
writer = pd.ExcelWriter("Fund Price Checks.xlsx", engine='xlsxwriter')
df_dict = {}
missing_in_gfd1_df.to_excel(writer,"Additional in new feed", index=False)
missing_in_cfg_df.to_excel(writer,"Missing in new feed", index=False)
for col in cfg_cols[len(on):]:
    print(col+"\n")
    check=[]
    columns = []
    columns = on + [col+"_CFG", col+"_GFD-1"]
    #print(columns)
    check_df = df.loc[:, columns]
    for n in range(df.shape[0]):
        if str(check_df.iloc[n, -2]) == str(check_df.iloc[n, -1]):
            check.append("Match")
        elif str(check_df.iloc[n, -2]) == "-":
            check.append("Price not in CFG")
        elif str(check_df.iloc[n, -1]) == "-":
            check.append("Price not in GFD-1")
        else:
            check.append("Difference")
    check_df["Check"] = np.array(check)
    check_df.to_excel(writer,'%s' % col, index=False)
    display(check_df.sample(n=5))
writer.save()

PriceCurrency



,LipperID,PriceDate,PriceCode,PriceType,PriceCurrency_CFG,PriceCurrency_GFD-1,Check
3398979,68193032,2017-07-20,D,N,CHF,CHF,Match
4785383,68316391,2017-07-27,D,N,GBP,GBP,Match
573846,60061868,2016-11-04,L,B,EUR,EUR,Match
3313598,68186003,2017-06-01,D,N,EUR,EUR,Match
3002706,68156012,2017-05-01,D,N,EUR,EUR,Match


PriceValueLC

